In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-02-18 00:06:59.176378


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [24]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [27]:
current_date.to_datetime64()

numpy.datetime64('2025-02-07T00:00:00.000000000')

In [4]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [5]:
fetch_data_to

Timestamp('2025-02-18 00:06:59.176378')

In [6]:
fetch_data_from

Timestamp('2025-01-20 00:06:59.176378')

In [7]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [8]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-01.
Loading data for 2024-01...
Total records: 2,964,624
Valid records: 2,911,483
Records dropped: 53,141 (1.79%)
Successfully processed data for 2024-01.
Combining all monthly data...
Data loading and processing complete!
Successfully downloaded data for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!


In [9]:
rides

,pickup_datetime,pickup_location_id
2265449,2025-02-12 16:25:44,2
2572150,2025-02-15 16:56:40,2
939566,2025-01-20 07:53:24,3
19476,2025-01-20 10:33:31,3
23806,2025-01-20 11:40:12,3
...,...,...
2766102,2025-02-17 23:40:22,263
2764349,2025-02-17 23:48:46,263
2764362,2025-02-17 23:54:59,263
2766206,2025-02-18 00:05:37,263


In [10]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [11]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-01-20 00:00:00,2,0
1,2025-01-20 01:00:00,2,0
2,2025-01-20 02:00:00,2,0
3,2025-01-20 03:00:00,2,0
4,2025-01-20 04:00:00,2,0
...,...,...,...
176336,2025-02-17 20:00:00,263,71
176337,2025-02-17 21:00:00,263,52
176338,2025-02-17 22:00:00,263,43
176339,2025-02-17 23:00:00,263,18


In [12]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176341 entries, 0 to 176340
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         176341 non-null  datetime64[ns]
 1   pickup_location_id  176341 non-null  int16         
 2   rides               176341 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [13]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-02-17 19:08:17,039 INFO: Initializing external client
2025-02-17 19:08:17,040 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-17 19:08:18,015 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213634


In [19]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |███████████████████████| Rows 176784/176784 | Elapsed Time: 00:43 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1211551/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)